In [1]:
%%time
import sys
if not r'G:\PythonProjects\WineRecognition2' in sys.path:
    sys.path.insert(0, r'G:\PythonProjects\WineRecognition2')
import os
import pickle
import json
import torch
import pandas as pd
import numpy as np
from data_master import DataGenerator, DataLoader, count_unk_foreach_tag, compute_model_confidence
from mlflow_utils import log_mlflow_on_test
from features import features

Wall time: 2.05 s


In [2]:
%%time
MODEL_PATH = r"G:/PythonProjects/WineRecognition2/artifacts/train/CRF_with_LSTM_and_Our_features_26112021_150906"
DATASET_PATH = r"G:\PythonProjects\WineRecognition2\data\text\menu_txt.txt"
LSTM_MODEL_PATH = r"G:/PythonProjects/WineRecognition2/artifacts/train/BiLSTM_CRF_24112021_223320/model/data/model.pth"
VOCAB_PATH = r"G:\PythonProjects\WineRecognition2\data\vocabs\Words_Halliday_Wine_AU.json"
CASE_SENSITIVE_VOCAB = False
DICTIONARY_PATH = r"G:\PythonProjects\WineRecognition2\data\dictionaries\Dict-byword_Halliday_Winesearcher_Wine_AU-only_completed_rows"
DATAINFO_PATH = 'G:/PythonProjects/WineRecognition2/data_info.json'
DO_PREPROCESS = False
DEVICE = 'cuda'
OUTPUT_DIR = ''
START_TIME = ''
RUN_NAME = 'test_run'
COMPUTE_METRICS = True

Wall time: 0 ns


In [3]:
%%time
with open(os.path.join(MODEL_PATH, 'model', 'model.pkl'), 'rb') as file:
    model = pickle.load(file)
freq_dict = DataLoader.load_frequency_dictionary(DICTIONARY_PATH)
bilstm_crf = torch.load(LSTM_MODEL_PATH).to(DEVICE).eval()
with open(VOCAB_PATH, 'r', encoding='utf-8') as file:
    word_to_ix = json.load(file)
if not CASE_SENSITIVE_VOCAB:
    word_to_ix = {word.lower(): index for word, index in word_to_ix.items()}

Wall time: 1.68 s


In [4]:
%%time
# getting features from lstm

def get_lstm_features(model, x):
    x = model.embedding(x)
    x, _ = model.lstm(x)
    return x

def features_with_keys(sentence):
    return [{f'A{i}': feature for i, feature in enumerate(features)} for features in sentence]

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def preprocess_sent(x_sent, y_sent):
    indices = [i for i, (x, y) in enumerate(zip(x_sent, y_sent)) if y == 'Add_BottleSize' and is_number(x)]
    for i in indices:
        x_sent[i] = str(float(x_sent[i]))

unk = 'UNK' if CASE_SENSITIVE_VOCAB else 'unk'
X_tensors = []

with torch.no_grad():
    sents = DataGenerator.generate_sents2(open(DATASET_PATH, encoding='utf-8').read().split('\n'))
    metadata = {'features': [], 'labels': []}

    for x_sent, y_sent in sents:
        if DO_PREPROCESS:
            preprocess_sent(x_sent, y_sent)
        
        our_features = features.sent2features(list(zip(x_sent, y_sent)), freq_dict)
        
        if not CASE_SENSITIVE_VOCAB:
            x_sent = [word.lower() for word in x_sent]
        
        x_tensor = torch.tensor(
            [word_to_ix[word] if word in word_to_ix else word_to_ix[unk] for word in x_sent],
            dtype=torch.int64
        )
        X_tensors.append(x_tensor)
        
        final_features = get_lstm_features(bilstm_crf, x_tensor.to(DEVICE).unsqueeze(0))
        final_features = features_with_keys(final_features.squeeze(0).detach().cpu().numpy())
        for i in range(len(x_sent)):
            final_features[i].update(our_features[i])
        metadata['features'].append(final_features)
        metadata['labels'].append(y_sent)
        
y_test = metadata['labels']
X_test = metadata['features']
len(X_test), len(y_test)

Wall time: 7.28 s


(43, 43)

In [5]:
%%time
y_pred = model.predict(X_test)
marginals = model.predict_marginals(X_test)

Wall time: 44.2 ms


In [6]:
with open(DATAINFO_PATH) as file:
    unk_foreach_tag = count_unk_foreach_tag(X_tensors, y_test, json.load(file)['keys']['all'], word_to_ix[unk])

In [7]:
confs = compute_model_confidence(marginals)
prob_table = DataGenerator.generate_probability_table(marginals, sents)

In [8]:
%%time
if COMPUTE_METRICS:
    test_eval = [list(zip(sentence, tags, y_pred[index])) for index, (sentence, tags) in enumerate(sents)]  
else:
    test_eval = []
    for i, (sentence, tags) in enumerate(sents):
        dct = dict.fromkeys(model.classes_, '')
        for j, word in enumerate(sentence):
            if y_pred[i][j] in dct.keys():
                dct[y_pred[i][j]] += f'{word} '
        test_eval.append({key: value.rstrip() for key, value in dct.items()})
    test_eval = pd.DataFrame({key: [wine.get(key) for wine in test_eval] for key in model.classes_})

Wall time: 0 ns


In [9]:
%%time
run_params = {
    'model_path': MODEL_PATH,
    'lstm_model_path': LSTM_MODEL_PATH,
    'vocab_path': VOCAB_PATH,
    'case_sensitive_vocab': CASE_SENSITIVE_VOCAB,
    'dictionary_path': DICTIONARY_PATH,
    'datainfo_path': DATAINFO_PATH,
    'device': DEVICE,
    'dataset_path': DATASET_PATH,
    'output_dir': OUTPUT_DIR,
    'compute_metrics': COMPUTE_METRICS,
    'start_time': START_TIME,
    'do_preprocess': DO_PREPROCESS,
    'runname': RUN_NAME,
    'models_confidence': np.mean(confs),
    'unk_foreach_tag': json.dumps(unk_foreach_tag),
    'prob_table': prob_table
}

Wall time: 0 ns


In [ ]:
log_mlflow_on_test(
    run_params=run_params,
    model=model,
    y_true=y_test,
    y_pred=y_pred,
    test_eval=test_eval
)